# U.S. Geological Survey Class GW3099
Advanced Modeling of Groundwater Flow (GW3099)\
Boise, Idaho\
September 16 - 20, 2024

![title](../../images/ClassLocation.jpg)

# Notebook that demonstrates some thing

Maybe a description of the problem

In [ ]:
import pathlib as pl

import flopy
import matplotlib.pyplot as plt
import numpy as np

## Heading

A section

## Heading

Another section

### Subheading